In [1]:
import os
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from datetime import datetime


In [2]:
cwd = os.getcwd()

# Convert the data into Python dataframe.
finance_dataset = pd.read_csv(f"{cwd}/financial_news.csv", usecols=['Date', 'Title'])

# total number of rows, each attribute type and number of non null values
finance_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50363 entries, 0 to 50362
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    50363 non-null  object
 1   Title   50363 non-null  object
dtypes: object(2)
memory usage: 787.0+ KB


In [3]:
date_unique = finance_dataset['Date'].unique()
columns=['Date', 'Summary']
df1 = pd.DataFrame(columns = columns)

for i in date_unique:
    value = finance_dataset.loc[finance_dataset['Date'] == i]
    summary = value.loc[:,'Title'].tolist()
    summary = ' '.join([str(i) for i in summary])

    list_d = [value['Date'].iloc[0], summary]
    newrows = pd.DataFrame([list_d], columns=columns)
    df1 = pd.concat([df1, newrows])

df1.to_csv(f"{cwd}/summarised_news.csv" ,header=columns, index=False)

In [4]:
df_financial_news = pd.read_csv(f"{cwd}/summarised_news.csv", usecols=columns)
df_financial_stock = pd.read_csv(f"{cwd}/2000-2019.csv", usecols=['Date', 'Close'])

def dmy_to_ymd(d):
    return datetime.strptime(d, '%d %b %Y').strftime('%Y-%m-%d')


for index, row in df_financial_stock.iterrows():
    converted_date = dmy_to_ymd(row.Date)
    df_financial_stock['Date'] = df_financial_stock['Date'].replace([row.Date], converted_date)

## Method 1
df_financial_stock['Label'] = df_financial_stock['Close'].mask(df_financial_stock.Close > df_financial_stock.Close.shift(-1), 1)
df_financial_stock['Label'] = df_financial_stock['Label'].mask(df_financial_stock.Close <= df_financial_stock.Close.shift(-1), 0)
df_financial_stock['Label'].values[-1:]=0

## Method 2
# rise_value = df_financial_news['Close'].shift(-1) + (0.001*df_financial_news['Close'].shift(-1))
# df_financial_news['Label'] = np.where(df_financial_news['Close'] > rise_value, 1, 0)

##Merge dataframes
df_financial_stock=df_financial_stock[['Date', 'Label', 'Close']]
output1 = pd.merge(df_financial_news, df_financial_stock, 
                   on='Date', 
                   how='inner')

print(output1)
print('\n\n')
print(output1['Label'].value_counts())
output1.to_csv(f"{cwd}/labelled_news.csv")


            Date                                            Summary  Label  \
0     2019-12-31  Third-party ATM switch ASPs to comply with cyb...    0.0   
1     2019-12-30  MDR waiver to hit fintechs, payments companies...    0.0   
2     2019-12-27  Impact of NBFC failures on banking system has ...    1.0   
3     2019-12-26  Airtel Payments Bank enables NEFT, users can m...    0.0   
4     2019-12-24  UCBs need to address weak governance, slow tec...    0.0   
...          ...                                                ...    ...   
4171  2003-02-17  Is The Honeymoon In The Debt Market Over? A Ch...    1.0   
4172  2003-02-14  Panel Report On Overhaul Of Recovery Of Debts ...    0.0   
4173  2003-02-12  Uti Bank Eyes Pref Issue As Tata Plan Hits Roa...    1.0   
4174  2003-02-11  Crisil Scripts Risk Gauge Tool For Film Indust...    0.0   
4175  2003-02-10  Net Scheduled Inflows Of Rs 1,559.9 Crore Rbi ...    0.0   

         Close  
0     32161.65  
1     32354.90  
2     32412.